# Neresbot 0.1

Modelos:
* Convolução
* Long Short-Term Memory
* SARIMA

In [2]:
import pandas as pd
import requests
import json
from pprint import pprint
import yahooquery as yq
import os

In [3]:
download = True
indexing = False

# Download dos Dados

Os símbolos dos tickers são obtidos manualmente do site: https://www.dadosdemercado.com.br/bolsa/acoes

In [4]:
tickers = pd.read_csv("acoes-listadas.csv")

In [5]:
tickers

,Código,Nome
0,PETR4,Petrobras
1,MGLU3,Magazine Luiza
2,HAPV3,Hapvida
3,CIEL3,Cielo
4,B3SA3,B3
...,...,...
522,TKNO3,Tekno
523,COCE6,Coelce
524,MGEL3,Mangels
525,CTSA8,Santanense


https://yahooquery.dpguthrie.com/

É criado um dicionário que conterá, como chave, o símbolo do ticker e, como valor, o dataframe relacionando medidas e períodos (dias).

Em cada dataframe é adicionado o devido dia da semana, numericamente.

In [6]:
# Os símbolos das empresas brasileiras recebem o sufixo ".SA".

símbolos = tickers['Código'] + '.SA'

In [7]:
# Verificando símbolos inválidos.

print(yq.Ticker(símbolos, backoff_factor=1, progress=True, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True).invalid_symbols)

None


Ajuste dos dados:
* Download e armazenamento:
  * Os índices são resetados
  * É verificado se os dados possuem a coluna 'date':
    * É criada a coluna equivalente aos dias semanais
    * Armazenamento no dict dados1
    * Os dados são salvos na pasta 'dados'

In [8]:
test = pd.DataFrame({'a':[1,2,3,4,5]})

In [9]:
test['tester'] = 0 if 'tester' not in test.columns else test['tester']

In [10]:
test

,a,tester
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [11]:
# 2
# Download dos dados e alocação dos dataframes no dicionário "dados1"

if download:
    dados1 = {}
    falhas = []

    for ticker in símbolos:
        query = yq.Ticker(ticker, backoff_factor=0.1, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)
        try:
            histórico = query.history(period='1y', interval='1d',adj_timezone=True)
        except:
            falhas.append(ticker)
            print(ticker,"falhou.")
        else:
            histórico = histórico.reset_index()
            if 'date' in histórico.columns: # Ajusta os tempos e salva em dados1
                histórico['date'] = pd.to_datetime(histórico['date'])
                histórico['weekday'] = histórico['date'].dt.weekday # Cria dias da semana
                histórico['date'] = histórico['date'].dt.tz_localize(None) # Remove timezone
                histórico['date'] = histórico['date'].dt.date # Remove horário
                histórico = histórico.sort_values('date') # Organiza por data
                
                dados1[ticker] = histórico # Salva tudo em dados1
            else:
                print(ticker,"não contém datas, mas contém:")
                print(histórico.columns)

    print(len(dados1),"tickers salvos com sucesso.")
    print(len(falhas),"tickers apresentaram falha")

    Medidas = []
    for tckr in dados1:
        for col in dados1[tckr].columns:
            if col not in Medidas:
                Medidas.append(col)
                print(col)
    for tckr in dados1:
        for col in Medidas:
            if col not in dados1[tckr].columns:
                dados1[tckr][col] = 0

    # Verifica se a pasta de destino existe, se não, cria-a
    if not os.path.exists('dados'):
        os.makedirs('dados')
    # Itera pelo dicionário e salva cada dataframe como um arquivo CSV
    for tckr, df in dados1.items():
        caminho_arquivo = os.path.join('dados', f'{tckr}.csv')  # Caminho completo para o arquivo CSV
        df.to_csv(caminho_arquivo, index=True)  # Salva o dataframe como arquivo CSV
        
        print(f'Arquivo {tckr}.csv salvo com sucesso.')

WIZS3.SA falhou.
LLIS3.SA falhou.
BRML3.SA falhou.
DMMO3.SA falhou.
GETT3.SA falhou.
GETT4.SA falhou.
SULA4.SA falhou.
SULA3.SA falhou.
CEPE5.SA falhou.
CEPE6.SA falhou.
BKBR3.SA falhou.
MTIG4.SA falhou.
BLUT4.SA falhou.
BLUT3.SA falhou.
MODL4.SA falhou.
CARD3.SA falhou.
FNCN3.SA falhou.
LCAM3.SA falhou.
BIDI4.SA falhou.
BIDI3.SA falhou.
EEEL4.SA falhou.
EEEL3.SA falhou.
BBRK3.SA falhou.
CESP6.SA falhou.
CESP3.SA falhou.
CESP5.SA falhou.
MOSI3.SA falhou.
POWE3.SA falhou.
GNDI3.SA falhou.
LAME4.SA falhou.
LAME3.SA falhou.
OMGE3.SA falhou.
IGTA3.SA falhou.
JPSA3.SA falhou.
BRDT3.SA falhou.
JBDU4.SA falhou.
JBDU3.SA falhou.
HGTX3.SA falhou.
CCPR3.SA falhou.
DTEX3.SA falhou.
VVAR3.SA falhou.
PNVL4.SA falhou.
TESA3.SA falhou.
BTOW3.SA falhou.
LINX3.SA falhou.
BTTL3.SA falhou.
GPCP3.SA falhou.
GPCP4.SA falhou.
SMLS3.SA falhou.
MMXM3.SA falhou.
BSEV3.SA falhou.
CNTO3.SA falhou.
TIET4.SA falhou.
TIET3.SA falhou.
CEPE3.SA falhou.
CALI4.SA falhou.
BPAR3.SA falhou.
APTI4.SA falhou.
CTSA8.SA não c

In [12]:
if not download:
    dados1 = {}  # Dicionário vazio para armazenar as tabelas

    # Percorre todos os arquivos na pasta
    for nome_arquivo in os.listdir('dados'):
        if nome_arquivo.endswith('.csv'):  # Verifica se o arquivo é um arquivo CSV
            caminho_arquivo = os.path.join('dados', nome_arquivo)  # Caminho completo para o arquivo

            histórico = pd.read_csv(caminho_arquivo)
            histórico['date'] = pd.to_datetime(histórico['date'])
            histórico['weekday'] = histórico['date'].dt.weekday # Cria dias da semana
            histórico['date'] = histórico['date'].dt.tz_localize(None) # Remove timezone
            histórico['date'] = histórico['date'].dt.date # Remove horário
            histórico = histórico.sort_values('date') # Organiza por data
            dados1[os.path.splitext(nome_arquivo)[0]] = histórico

    # Imprime o dicionário com as tabelas
    print(len(dados1))

    print(dados1['PETR4.SA'])

    for i in dados1:
        print(i)

In [13]:
# Verificando se todas as colunas 'date' são iguais 

ticker_divergentes = pd.DataFrame(columns=dados1.keys(), index=dados1.keys())

for column in dados1.keys():
    for index in dados1.keys():
        if dados1[column]['date'].equals(dados1[index]['date']):
            ticker_divergentes.at[index,column] = True
        else:
            ticker_divergentes.at[index,column] = False

ticker_divergentes.to_csv('ticker_divergentes.csv')
ticker_divergentes

,PETR4.SA,MGLU3.SA,HAPV3.SA,CIEL3.SA,B3SA3.SA,VIIA3.SA,ITSA4.SA,CVCB3.SA,VALE3.SA,BBDC4.SA,...,CASN4.SA,EMAE3.SA,VSPT3.SA,MTIG3.SA,FIGE4.SA,LUXM3.SA,TKNO3.SA,COCE6.SA,MGEL3.SA,MMAQ4.SA
PETR4.SA,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
MGLU3.SA,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
HAPV3.SA,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
CIEL3.SA,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
B3SA3.SA,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LUXM3.SA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
TKNO3.SA,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,True,True,True,True
COCE6.SA,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,True,True,True,True
MGEL3.SA,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,True,True,True,True


In [14]:
sets_by_date = []

for tckr in dados1:
    hist = dados1[tckr]
    # A relação chave:valor é de tckr:hist

    if len(sets_by_date) == 0:
        sets_by_date.append({})
    
    set_index = 0
    Verif = True

    while Verif:
        if len(sets_by_date[set_index]) == 0:
            sets_by_date[set_index][tckr] = hist
            sets_by_date[set_index]['hist'] = hist['date']
            Verif = False
        else:
            if sets_by_date[set_index]['hist'].equals(hist['date']):
                sets_by_date[set_index][tckr] = hist
                Verif = False
            else:
                set_index += 1
                if set_index == len(sets_by_date):
                    sets_by_date.append({})


Analisando: PETR4.SA
sets_by_date estava vazio.
tamanho do sets_by_date: 1
Analisando: MGLU3.SA
Analisando: HAPV3.SA
Analisando: CIEL3.SA
Analisando: B3SA3.SA
Analisando: VIIA3.SA
Analisando: ITSA4.SA
Analisando: CVCB3.SA
Analisando: VALE3.SA
Analisando: BBDC4.SA
Analisando: ITUB4.SA
Analisando: AMER3.SA
Analisando: ASAI3.SA
Analisando: TRAD3.SA
Analisando: BBAS3.SA
Analisando: OIBR3.SA
Analisando: PETR3.SA
Analisando: ABEV3.SA
Analisando: LREN3.SA
Analisando: CPLE6.SA
Analisando: COGN3.SA
Analisando: GOLL4.SA
Analisando: RAIZ4.SA
Analisando: RAIL3.SA
Analisando: SUZB3.SA
Analisando: INTB3.SA
Analisando: CSAN3.SA
Analisando: MRVE3.SA
Analisando: BRFS3.SA
Analisando: BRKM5.SA
Analisando: AZUL4.SA
Analisando: AMAR3.SA
Analisando: VBBR3.SA
Analisando: CMIN3.SA
Analisando: CEAB3.SA
Analisando: EQTL3.SA
Analisando: JBSS3.SA
Analisando: ELET3.SA
Analisando: SEQL3.SA
Analisando: POMO4.SA
Analisando: USIM5.SA
Analisando: YDUQ3.SA
Analisando: LIGT3.SA
Analisando: GGBR4.SA
Analisando: CCRO3.SA
A

# Convolução

## Rótulos

Cada linha analisa se há aumento expressivo nos próximos 5 períodos.

Se não houver, o rótulo será 0; Caso contrário, será o número de dias em que o aumento ocorre.  
Há uma variável ajustável para determinar o nível do aumento.

In [20]:
dados2 = sets_by_date[0]
del dados2['hist']

In [60]:
# Coluna para os rótulos

for i,j in dados2.items():
    j['rótulos'] = 0

In [61]:
# Metaparâmetros

Aumento = 0.1

Prospecção = 5

In [62]:
for key, df in dados2.items():                  # Para cada ticker, tabela:                      # reseta o índice da tabela
    for i, row in df.iterrows():                  # para cada índice, coluna da tabela:
        price_now = row['close']                     # o preço atualé o número contido na coluna 'close'

        price_contester = price_now
        for j in range(i+1, i+1+Prospecção):         # para cada número j entre o índice atual e o índice + prospecção (+1 pois o último desconsidera)
            if j < len(df):                             # se j é menor que o tamanho da tabela:
                post_high = df.loc[j, 'high']              # o preço avaliado é o o número no índice j, na coluna 'high'
                if (post_high >= (1 + Aumento)*price_now) and (post_high > price_contester):   # se o preço avaliado é maior ou igual ao aumento esperado:
                    dados2[key].loc[i, 'rótulos'] = j - i              # o valor em 'rótulo' é o índice em j menos o índice em i (que é os períodos à frente)
                    price_contester = post_high
                    if (j-i) > 0:
                        print(row['symbol'],row['date'],dados2[key].loc[i, 'rótulos'],(((post_high/price_now)-1)*100))
    
    # print(df.head()) #??

PETR4.SA 2022-07-21 5 12.439698667839917
PETR4.SA 2022-07-22 4 11.251282485486636
PETR4.SA 2022-07-22 5 18.17251939164879
PETR4.SA 2022-07-25 4 12.899019498569864
PETR4.SA 2022-07-26 3 11.77039532921631
PETR4.SA 2022-07-27 2 10.558211884102086
PETR4.SA 2022-08-02 5 10.730112494660782
PETR4.SA 2022-08-03 4 10.664699477347007
PETR4.SA 2022-08-03 5 11.226006567365076
PETR4.SA 2022-08-04 4 10.055546721623877
PETR4.SA 2022-08-04 5 10.669399659632205
PETR4.SA 2022-09-26 5 11.437345064558112
PETR4.SA 2022-09-27 4 10.010108883442491
PETR4.SA 2022-09-27 5 11.156048053356905
PETR4.SA 2022-09-28 3 11.513491216877991
PETR4.SA 2022-09-28 4 12.675090626753317
PETR4.SA 2022-09-29 2 11.36131007977481
PETR4.SA 2022-09-29 3 12.521324268730915
PETR4.SA 2022-09-29 5 15.284890928744744
PETR4.SA 2022-09-30 2 10.67114223687964
PETR4.SA 2022-09-30 4 13.389267720197328
PETR4.SA 2022-09-30 5 15.167787159344638
PETR4.SA 2022-10-14 5 14.871338997622608
PETR4.SA 2022-10-17 4 14.974543550156817
PETR4.SA 2022-10-18 

In [63]:
for tckr in dados2:
    print(dados2[tckr].tail(4))

       symbol        date       open       high        low      close  \
246  PETR4.SA  2023-06-14  28.940001  30.100000  28.879999  30.100000   
247  PETR4.SA  2023-06-15  30.200001  30.850000  29.280001  29.389999   
248  PETR4.SA  2023-06-16  29.230000  29.920000  29.040001  29.639999   
249  PETR4.SA  2023-06-19  29.629999  30.459999  29.549999  30.420000   

        volume   adjclose  dividends  weekday  splits  rótulos  
246   98326000  30.100000        0.0        2       0        0  
247  120579500  29.389999        0.0        3       0        0  
248   94572000  29.639999        0.0        4       0        0  
249   34453100  30.420000        0.0        0       0        0  
       symbol        date  open  high   low  close     volume  adjclose  \
246  MGLU3.SA  2023-06-14  3.81  3.84  3.69   3.81  135391300      3.81   
247  MGLU3.SA  2023-06-15  3.79  3.82  3.73   3.75  115498600      3.75   
248  MGLU3.SA  2023-06-16  3.74  3.79  3.70   3.71   87510800      3.71   
249  MGLU

In [64]:
print('  Dividendos')
for tckr in dados2:
    print(tckr+':',dados2[tckr]['dividends'].max())

  Dividendos
PETR4.SA: 6.732003
MGLU3.SA: 0
HAPV3.SA: 0
CIEL3.SA: 0.28084
B3SA3.SA: 0.130776
VIIA3.SA: 0
ITSA4.SA: 0.1855
CVCB3.SA: 0
VALE3.SA: 3.572057
BBDC4.SA: 0.583406
ITUB4.SA: 0.48618
AMER3.SA: 0
ASAI3.SA: 0.050019
TRAD3.SA: 0
BBAS3.SA: 0.826881
OIBR3.SA: 0
PETR3.SA: 6.732003
ABEV3.SA: 0.7623
LREN3.SA: 0.184049
CPLE6.SA: 0.367323
COGN3.SA: 0
GOLL4.SA: 0
RAIZ4.SA: 0.089066
RAIL3.SA: 0.066017
SUZB3.SA: 1.794781
INTB3.SA: 0.197381
CSAN3.SA: 0.42858
MRVE3.SA: 0.197808
BRFS3.SA: 0
BRKM5.SA: 0
AZUL4.SA: 0
AMAR3.SA: 0
VBBR3.SA: 0.390162
CMIN3.SA: 0.445689
CEAB3.SA: 0
EQTL3.SA: 0.35
JBSS3.SA: 1.0
ELET3.SA: 0.220929
SEQL3.SA: 0
POMO4.SA: 0.142
USIM5.SA: 0.328767
YDUQ3.SA: 0
LIGT3.SA: 0
GGBR4.SA: 2.047619
CCRO3.SA: 0.292863
EMBR3.SA: 0
NTCO3.SA: 0
PETZ3.SA: 0.010831
IFCM3.SA: 0
CMIG4.SA: 0.234269
RENT3.SA: 0.369177
MRFG3.SA: 0.909518
ANIM3.SA: 0
CBAV3.SA: 0.345806
GFSA3.SA: 0
CYRE3.SA: 0.511295
SOMA3.SA: 0.101443
CRFB3.SA: 0.128351
BBSE3.SA: 1.878577
RRRP3.SA: 0
CSNA3.SA: 1.17949
PRIO3.SA:

In [65]:
print('  Splits')
for tckr in dados2:
    print(tckr+':',dados2[tckr]['splits'].max())

  Splits
PETR4.SA: 0
MGLU3.SA: 0
HAPV3.SA: 0
CIEL3.SA: 0
B3SA3.SA: 0
VIIA3.SA: 0
ITSA4.SA: 1.1
CVCB3.SA: 0
VALE3.SA: 0
BBDC4.SA: 0
ITUB4.SA: 0
AMER3.SA: 0
ASAI3.SA: 0
TRAD3.SA: 0
BBAS3.SA: 0
OIBR3.SA: 0.1
PETR3.SA: 0
ABEV3.SA: 0
LREN3.SA: 0
CPLE6.SA: 0
COGN3.SA: 0
GOLL4.SA: 0
RAIZ4.SA: 0
RAIL3.SA: 0
SUZB3.SA: 0
INTB3.SA: 0
CSAN3.SA: 0
MRVE3.SA: 0
BRFS3.SA: 0
BRKM5.SA: 0
AZUL4.SA: 0
AMAR3.SA: 0
VBBR3.SA: 0
CMIN3.SA: 0
CEAB3.SA: 0
EQTL3.SA: 0
JBSS3.SA: 0
ELET3.SA: 0
SEQL3.SA: 0
POMO4.SA: 0
USIM5.SA: 0
YDUQ3.SA: 0
LIGT3.SA: 0
GGBR4.SA: 1.05
CCRO3.SA: 0
EMBR3.SA: 0
NTCO3.SA: 0
PETZ3.SA: 0
IFCM3.SA: 0
CMIG4.SA: 0
RENT3.SA: 0
MRFG3.SA: 0
ANIM3.SA: 0
CBAV3.SA: 0
GFSA3.SA: 0.1111111111111111
CYRE3.SA: 0
SOMA3.SA: 0
CRFB3.SA: 0
BBSE3.SA: 0
RRRP3.SA: 0
CSNA3.SA: 0
PRIO3.SA: 0
LWSA3.SA: 0
BBDC3.SA: 0
WEGE3.SA: 0
NEOE3.SA: 0
ALPA4.SA: 0
TIMS3.SA: 0
TOTS3.SA: 0
GMAT3.SA: 0
ENEV3.SA: 0
GOAU4.SA: 0
ECOR3.SA: 0
UGPA3.SA: 0
FLRY3.SA: 1.05
VAMO3.SA: 0
QUAL3.SA: 0
RADL3.SA: 1.04
MULT3.SA: 0
RDOR3.SA: 0
B

In [66]:
print('  Rótulos')
for tckr in dados2:
    print(tckr+':',dados2[tckr]['rótulos'].max())

  Rótulos
PETR4.SA: 5
MGLU3.SA: 5
HAPV3.SA: 5
CIEL3.SA: 5
B3SA3.SA: 5
VIIA3.SA: 5
ITSA4.SA: 5
CVCB3.SA: 5
VALE3.SA: 5
BBDC4.SA: 5
ITUB4.SA: 5
AMER3.SA: 5
ASAI3.SA: 5
TRAD3.SA: 5
BBAS3.SA: 5
OIBR3.SA: 5
PETR3.SA: 5
ABEV3.SA: 0
LREN3.SA: 5
CPLE6.SA: 5
COGN3.SA: 5
GOLL4.SA: 5
RAIZ4.SA: 5
RAIL3.SA: 5
SUZB3.SA: 5
INTB3.SA: 5
CSAN3.SA: 5
MRVE3.SA: 5
BRFS3.SA: 5
BRKM5.SA: 5
AZUL4.SA: 5
AMAR3.SA: 5
VBBR3.SA: 5
CMIN3.SA: 5
CEAB3.SA: 5
EQTL3.SA: 5
JBSS3.SA: 5
ELET3.SA: 5
SEQL3.SA: 5
POMO4.SA: 5
USIM5.SA: 5
YDUQ3.SA: 5
LIGT3.SA: 5
GGBR4.SA: 5
CCRO3.SA: 5
EMBR3.SA: 5
NTCO3.SA: 5
PETZ3.SA: 5
IFCM3.SA: 5
CMIG4.SA: 5
RENT3.SA: 5
MRFG3.SA: 5
ANIM3.SA: 5
CBAV3.SA: 5
GFSA3.SA: 5
CYRE3.SA: 5
SOMA3.SA: 5
CRFB3.SA: 5
BBSE3.SA: 5
RRRP3.SA: 5
CSNA3.SA: 5
PRIO3.SA: 5
LWSA3.SA: 5
BBDC3.SA: 5
WEGE3.SA: 5
NEOE3.SA: 5
ALPA4.SA: 5
TIMS3.SA: 5
TOTS3.SA: 5
GMAT3.SA: 5
ENEV3.SA: 5
GOAU4.SA: 5
ECOR3.SA: 5
UGPA3.SA: 5
FLRY3.SA: 5
VAMO3.SA: 5
QUAL3.SA: 5
RADL3.SA: 5
MULT3.SA: 5
RDOR3.SA: 5
BEEF3.SA: 5
POSI3.SA: 5
ODPV3.

## Transformação das matrizes: Separando os dias como objetos numpy.

Classes que contenham as matrizes, os rótulos e a identificação do dia.

In [67]:
# Todos os dataframes:

df_concatenado = pd.concat(dados2.values(), axis=1)

In [68]:
df_concatenado.to_csv('df_concatenado.csv',index=True)

O último dia não está incluso no treinamento por não possuir rótulo.

Dimensões:
* Medidas:
  * Dia da semana
  * Dia do ano
  * Abertura
  * Mínimo
  * Máximo
  * Fechamento
  * Fechamento Ajustado
  * Volume
* tickers
* Dias (atual e anteriores)

Há uma variável ajustável para determinar quantos dias serão olhados para trás.

Um dicionário de rótulos, um rótulo para cada ticker, é atrelado a cada matriz.

## Separação entre Treino, Teste e Validação.

É preciso garantir que as matrizes sejam embaralhadas, mas que os rótulos permaneçam atrelados adequadamente.

## Modelo

* Para cada ticker (a partir do dicionário de rótulos):  
    * O modelo é treinado seguindo aquele ticker como rótulo (com uma função)  
    * O modelo é validado  
    * Se o modelo é válido:  
        * A previsão é tentada:  
        * Se o resultado é diferente de zero E a probabilidade é alta:  
                * O deploy é salvo